# 1. Exploratory data analysis

In [80]:
import carto2gpd
import cenpy
import dask.dataframe as dd
import numpy as np
import pandas as pd
import geopandas as gpd
import spatialpandas as spd
import datetime as dt
import missingno as msno

import xarray as xr
import hvplot.xarray
import hvplot.pandas
import holoviews as hv
from holoviews.operation.datashader import datashade, spread
from datashader.utils import lnglat_to_meters
import geoviews as gv
import panel as pn

from io import BytesIO
import gzip
from urllib.request import urlopen

## Data wrangling

As stated in the beginning, the LODES data structure is both high-resolution and quite longitudinal; the Census geography is the block, the smallest possible, but ACS data is only up to the block group level, so fortunately we can just aggregate by the first 12 digits of the FIPS code for each block. Temporally, data stretches back to 2002, though as 2009 was the start of higher-quality data collection according to the Census Bureau, we begin with that year instead.

The LODES database consists of three separate datasets. These consist of:

- Workplace Area Characteristics (WAC): Per block, count of the workplaces of workers. Crosstabs include age group, income group, NAICS industry sector (construction, finance, etc.), race, education, age and size of firm.
- Residence Area Characteristics (RAC): Per block, count of the residences of workers. Crosstabs are the same as WAC
- Origin-Destination (OD): Per worker, denote their residence block, workplace block, and age group/income group/basic industry sector (goods producing, trade/transportation, and others)

This section is concerned with WAC and RAC datasets, and basic inferences on their content. We use built-in Python functions to directly download from the Census Bureau FTP server, and as data is split by year, aggregate years 2009-2019 together in one Dask DataFrame.

In [2]:
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

In [3]:
url = "https://lehd.ces.census.gov/data/lodes/LODES7/pa/wac/pa_wac_S000_JT00_"
wac = dd.concat([pd.read_csv(gzip.open(BytesIO(urlopen(url+year+".csv.gz").read()))).assign(year=int(year)) for year in years])
wac['w_geocode'] = wac['w_geocode'].astype('str').str[:12]
wac.head()

,w_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,...,CFA03,CFA04,CFA05,CFS01,CFS02,CFS03,CFS04,CFS05,createdate,year
0,420010301011,15,1,14,0,3,9,3,14,0,...,0,0,0,0,0,0,0,0,20160228,2009
1,420010301011,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,20160228,2009
2,420010301011,54,9,32,13,17,29,8,50,0,...,0,0,0,0,0,0,0,0,20160228,2009
3,420010301011,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,20160228,2009
4,420010301011,25,5,15,5,20,5,0,0,0,...,0,0,0,0,0,0,0,0,20160228,2009


In [4]:
url = "https://lehd.ces.census.gov/data/lodes/LODES7/pa/rac/pa_rac_S000_JT00_"
rac = dd.concat([pd.read_csv(gzip.open(BytesIO(urlopen(url+year+".csv.gz").read()))).assign(year=int(year)) for year in years])
rac['h_geocode'] = rac['h_geocode'].astype('str').str[:12]
rac.head()

,h_geocode,C000,CA01,CA02,CA03,CE01,CE02,CE03,CNS01,CNS02,...,CT01,CT02,CD01,CD02,CD03,CD04,CS01,CS02,createdate,year
0,420010301011,4,0,3,1,1,3,0,0,0,...,4,0,2,2,0,0,2,2,20160228,2009
1,420010301011,12,1,7,4,4,4,4,0,0,...,12,0,0,6,2,3,4,8,20160228,2009
2,420010301011,11,3,7,1,2,6,3,1,0,...,11,0,1,4,1,2,4,7,20160228,2009
3,420010301011,43,13,18,12,16,14,13,1,1,...,43,0,2,11,5,12,19,24,20160228,2009
4,420010301011,14,4,7,3,0,5,9,0,0,...,14,0,0,3,5,2,8,6,20160228,2009


To limit our selected area of analysis, we load Census TIGER block group shapefiles for Philadelphia and its five surrounding PA counties. Including New Jersey and Delaware block groups as well as LODES data is possible, but the size of the data already was at the limit for the available computing power.

In [5]:
phl_bg = gpd.read_file("bg/tl_2019_42_bg/tl_2019_42_bg.shp").query("COUNTYFP in ['017','029','045','091','101']").to_crs('EPSG:3857')

### Viz 1: Hexbin of workplace and residence locations

Simply plotting the overall density of workplaces and residences will give us a good idea of the kind of inherent spatial structure we are dealing with in the Philadelphia region. After summing and merging each block group, we take the centroid of each block group, as `hvPlot` only recognizes lat/long for hexbins.

In [6]:
wac_bg_sum = wac.groupby(['year','w_geocode'])['C000'].sum().compute().reset_index()
wac_bg_sum = gpd.GeoDataFrame(phl_bg[['GEOID','geometry']].merge(wac_bg_sum, left_on='GEOID', right_on='w_geocode', how='inner'),
            geometry='geometry',crs='EPSG:3857')
wac_bg_sum['Longitude'] = wac_bg_sum.to_crs(4326).geometry.centroid.x
wac_bg_sum['Latitude'] = wac_bg_sum.to_crs(4326).geometry.centroid.y
wac_bg_sum['C000'].isnull().values.any()
wac_bg_sum

/var/folders/sp/29cgd4lj47q9vf36fnw27d780000gn/T/ipykernel_63222/2439271376.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wac_bg_sum['Longitude'] = wac_bg_sum.to_crs(4326).geometry.centroid.x
/var/folders/sp/29cgd4lj47q9vf36fnw27d780000gn/T/ipykernel_63222/2439271376.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wac_bg_sum['Latitude'] = wac_bg_sum.to_crs(4326).geometry.centroid.y


,GEOID,geometry,year,w_geocode,C000,Longitude,Latitude
0,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2009,420293035021,252,-75.746319,39.880268
1,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2010,420293035021,255,-75.746319,39.880268
2,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2011,420293035021,264,-75.746319,39.880268
3,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2012,420293035021,246,-75.746319,39.880268
4,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2013,420293035021,277,-75.746319,39.880268
...,...,...,...,...,...,...,...
32634,420293044043,"POLYGON ((-8422751.833 4872425.194, -8422743.2...",2015,420293044043,201,-75.656472,40.048608
32635,420293044043,"POLYGON ((-8422751.833 4872425.194, -8422743.2...",2016,420293044043,228,-75.656472,40.048608
32636,420293044043,"POLYGON ((-8422751.833 4872425.194, -8422743.2...",2017,420293044043,232,-75.656472,40.048608
32637,420293044043,"POLYGON ((-8422751.833 4872425.194, -8422743.2...",2018,420293044043,226,-75.656472,40.048608


In [7]:
rac_bg_sum = rac.groupby(['year','h_geocode'])['C000'].sum().compute().reset_index()
rac_bg_sum = gpd.GeoDataFrame(phl_bg[['GEOID','geometry']].merge(rac_bg_sum, left_on='GEOID', right_on='h_geocode', how='inner'),
            geometry='geometry',crs='EPSG:3857')
rac_bg_sum['Longitude'] = rac_bg_sum.to_crs(4326).geometry.centroid.x
rac_bg_sum['Latitude'] = rac_bg_sum.to_crs(4326).geometry.centroid.y
rac_bg_sum['C000'].isnull().values.any()
rac_bg_sum

/var/folders/sp/29cgd4lj47q9vf36fnw27d780000gn/T/ipykernel_63222/2232090582.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  rac_bg_sum['Longitude'] = rac_bg_sum.to_crs(4326).geometry.centroid.x
/var/folders/sp/29cgd4lj47q9vf36fnw27d780000gn/T/ipykernel_63222/2232090582.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  rac_bg_sum['Latitude'] = rac_bg_sum.to_crs(4326).geometry.centroid.y


,GEOID,geometry,year,h_geocode,C000,Longitude,Latitude
0,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2009,420293035021,688,-75.746319,39.880268
1,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2010,420293035021,754,-75.746319,39.880268
2,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2011,420293035021,593,-75.746319,39.880268
3,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2012,420293035021,553,-75.746319,39.880268
4,420293035021,"POLYGON ((-8435115.268 4853101.891, -8435096.6...",2013,420293035021,581,-75.746319,39.880268
...,...,...,...,...,...,...,...
32759,420293044043,"POLYGON ((-8422751.833 4872425.194, -8422743.2...",2015,420293044043,582,-75.656472,40.048608
32760,420293044043,"POLYGON ((-8422751.833 4872425.194, -8422743.2...",2016,420293044043,586,-75.656472,40.048608
32761,420293044043,"POLYGON ((-8422751.833 4872425.194, -8422743.2...",2017,420293044043,567,-75.656472,40.048608
32762,420293044043,"POLYGON ((-8422751.833 4872425.194, -8422743.2...",2018,420293044043,573,-75.656472,40.048608


We see that while the workplace hexbin on the left does show a few hexes with high density, the residential hexbin on right is more or less uniform across the entire region, and this is not a mistake as the colors do change slightly over years. Later we will see if the OD dataset and using raw point data is perhaps more reliable to visualize residential density.

In [75]:
hv_panel = pn.panel(hv.Layout(
wac_bg_sum.hvplot.hexbin(title='Workplace hexbin',x='Longitude',y='Latitude',geo=True,c='C000',tiles='StamenTonerBackground',hover_cols='C000',groupby='year').map(gv.HexTiles,hv.HexTiles) \
    .options(hv.opts.HexTiles(alpha=0.7,aggregator=np.mean,width=800,height=800,xaxis=None,yaxis=None)) +
rac_bg_sum.hvplot.hexbin(title='Residence hexbin',x='Longitude',y='Latitude',geo=True,c='C000',tiles='StamenTonerBackground',hover_cols='C000',groupby='year').map(gv.HexTiles,hv.HexTiles) \
    .options(hv.opts.HexTiles(alpha=0.7,aggregator=np.mean,width=800,height=800,xaxis=None,yaxis=None))
).cols(2))

widgets=hv_panel[1]

comb = pn.Column(
    pn.Row(*widgets),
    hv_panel[0])

comb.save('./charts/1_hexbin.html',embed=True)

The following was an attempt to use `xarray` and hvPlot's `contourf` 2d kernel density function, but this was not successful.

In [9]:
wac_bg_xr = pd.DataFrame(wac_bg_sum.drop(columns=['GEOID','geometry','w_geocode'])).set_index(['Longitude','Latitude','year']).to_xarray()
wac_bg_xr

<xarray.Dataset>
Dimensions:    (Longitude: 2979, Latitude: 2979, year: 11)
Coordinates:
  * Longitude  (Longitude) float64 -76.05 -76.01 -75.99 ... -74.78 -74.77 -74.76
  * Latitude   (Latitude) float64 39.73 39.74 39.74 39.75 ... 40.55 40.57 40.6
  * year       (year) int64 2009 2010 2011 2012 2013 ... 2016 2017 2018 2019
Data variables:
    C000       (Longitude, Latitude, year) float64 nan nan nan ... nan nan nan

In [10]:
wac_bg_xr.hvplot.contourf(
    x='Longitude',y='Latitude',z='C000',groupby='year',levels=8)

:DynamicMap   [year]
   :Polygons   [Longitude,Latitude]   (C000)

### Viz 2: Comparing worker job locations to residence locations

Since every block group has both workplaces and residences, it is worthwhile to see how they compare head-to-head. The left plot simply reflects if the block group has a greater raw count of workplaces or residences, while the second is a ratio; the second actually is relatively close to the algorithm Manduca used to determine central business districts, as he found was more reliable and versatile in different urban contexts.
We can see that the first plot casts a much wider net on the eligibility for a workplace-characterized tract than the second. Later on we will use clustering to create a more sophisticated workplace-detection method.

In [89]:
comp = pd.merge(wac_bg_sum[['year','w_geocode','geometry','C000']],rac_bg_sum[['year','h_geocode','C000']],
                left_on=['year','w_geocode'],right_on=['year','h_geocode'],how='inner')
comp['net'] = np.where(comp['C000_x'] - comp['C000_y'] > 0,'More workplaces','More residences')
comp['ratio'] =comp['C000_x']/comp['C000_y']
comp = gpd.GeoDataFrame(comp[comp['ratio'] < 700])


# hv_panel = pn.panel(hv.Layout(
#     comp.hvplot(title="More workplaces or residents by block group",geo=True,crs=3857,groupby='year',c='net')
#     +
#     comp.hvplot(title="Ratio of workplaces to residents by block group",geo=True,crs=3857,groupby='year',c='ratio')
# ).cols(2))
# widgets=hv_panel[1]

# comb = pn.Column(
#     pn.Row(*widgets),
#     hv_panel[0])

hv.save(comp.hvplot(title="More workplaces or residents by block group",geo=True,crs=3857,groupby='year',c='net',widget_position='top'),'./charts/1_w_or_h.html',embed=True)
hv.save(comp.hvplot(title="Ratio of workplaces to residents by block group",geo=True,crs=3857,groupby='year',c='ratio',widget_position='top'),'./charts/1_wh_ratio.html',embed=True)

/Users/bensh/.pyenv/versions/miniconda3-latest/envs/musa-550-fall-2021/lib/python3.8/site-packages/spatialpandas/geometry/multipolygon.py:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  polygon_arrays = np.asarray(self.data.as_py())
Invoked as dynamic_operation(2009, height=400, scale=1.0, width=400, x_range=None, y_range=None)
Invoked as dynamic_operation(2009, height=400, scale=1.0, width=400, x_range=None, y_range=None)


ValueError: input must be numeric

Column
    [0] Row
        [0] WidgetBox
            [0] DiscreteSlider(formatter='%d', margin=(20, 20, 20, 20), name='year', options=OrderedDict([('2009', ...]), value=2009, width=250)
        [1] VSpacer()
    [1] HoloViews(Layout)

### Viz 3: Looking at jobs and broken by industry/income over time

Here we take the crosstabs for industry and income, and evaluate how they change over the 11-year period 2009-2019. We can see that while job sectors have barely changed over time except for an increase in healthcare jobs (not a surprise for Philadelphia), the highest income bracket has increased over time relative to the others.

In [63]:
allcol = [col for col in wac if col.startswith('C')]
cnscol = [col for col in wac if col.startswith('CNS')]
inccol = [col for col in wac if col.startswith('CE')]
wac_year_sum = wac.groupby('year')[allcol].sum().compute()
hv.save(wac_year_sum.hvplot.area(title='Jobs by industry, 2009-2019',
widget_location='top',x='year',y=cnscol,value_label='Job count'),'./charts/1_jobs_industry.html',embed=True)

In [64]:
hv.save(wac_year_sum.hvplot.area(title='Jobs by income bracket, 2009-2019',
widget_location='top',x='year',y=inccol,value_label='Job count'),'./charts/1_jobs_income.html',embed=True)

### Viz 4: Density-distance plot: Workplace and population as a function of distance from CBD

Next, we aggregate a level higher onto the census tract and merge with ACS population for 2013-2019. Then we calculate the distance in feet from each census tract centroid to Center City, and plot the results for workplace and population density by this distance.
We can see that while population density is relatively uniform by distance. job density peaks significantly at the CBD, as expected

In [46]:
pop_phl = pd.DataFrame()

for yr in [int(i) for i in years[4:]]:
    acs = cenpy.products.ACS(year=yr)
    pop_phl = pop_phl.append(acs.from_msa(msa='Philadelphia-Camden-Wilmington, PA-NJ-DE-MD', variables=["B01001_001E"], level='tract', return_geometry=True) \
        .assign(year=yr))

/Users/bensh/.pyenv/versions/miniconda3-latest/envs/musa-550-fall-2021/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/bensh/.pyenv/versions/miniconda3-latest/envs/musa-550-fall-2021/lib/python3.8/site-packages/cenpy/geoparser.py:226: UserWarning: Shape is invalid: 
Ring Self-intersection[-8334206.74 4941501.12]
  tell_user("Shape is invalid: \n{}".format(vexplain))
/Users/bensh/.pyenv/versions/miniconda3-latest/envs/musa-550-fall-2021/lib/python3.8/site-packages/cenpy/geoparser.py:226: UserWarning: Shape is invalid: 
Ring Self-intersection[-8339755.58 4964363.7]
  tell_user("Shape is invalid: \n{}".format(vexplain))
/Users/bensh/.pyenv/versions/miniconda3-latest/envs/m

In [50]:
from shapely.geometry import Point
wac_ct_sum = wac_bg_sum.drop(columns=['geometry','GEOID'])
wac_ct_sum['GEOID'] = wac_ct_sum['w_geocode'].astype('str').str[:11]
wac_ct_sum = wac_ct_sum.groupby(['year','GEOID'])['C000'].sum().reset_index()
wac_ct_sum = gpd.GeoDataFrame(pd.merge(wac_ct_sum,pop_phl[['year','GEOID','geometry','B01001_001E']],on=['year','GEOID'],how='inner'),geometry='geometry',crs='EPSG:3857')
wac_ct_sum['distCenterCity'] = wac_ct_sum['geometry'].centroid.distance(Point(-8368112.55, 4859373.64))
wac_ct_sum = wac_ct_sum.sort_values(by=['year','distCenterCity'])

wac_ct_sum['densJob'] = (wac_ct_sum['C000'] / wac_ct_sum['geometry'].area) *  2.788e+7
wac_ct_sum['densPop'] = (wac_ct_sum['B01001_001E'] / wac_ct_sum['geometry'].area) *  2.788e+7

hv.save(pd.DataFrame(wac_ct_sum).hvplot.scatter(title='Distance-density plot, Philadelphia region census tracts',
widget_location='top',x='distCenterCity',y=['densJob','densPop'],groupby='year'),"./charts/1_dens_distance.html",embed=True)

The same phenomenon can be found when plotting cumulative count of population and jobs by distance, where job count overtakes population count nearest Center City for up to 5000 feet; then the population rise overtakes and rises slightly faster than jobs, suggesting that population density still dominates over job density in the suburbs.

In [62]:
wac_ct_sum['cumulPop'] = wac_ct_sum.groupby(['year'])['B01001_001E'].cumsum()
wac_ct_sum['cumulJob'] = wac_ct_sum.groupby(['year'])['C000'].cumsum()

cum_distance = pd.DataFrame(wac_ct_sum).hvplot.scatter(title='Distance-cumulative total plot, Philadelphia region census tracts',
widget_location='top',x='distCenterCity',y=['cumulPop','cumulJob'],groupby='year')

cum_distance.save("./charts/1_cum_distance.html",embed=True)